<a href="https://colab.research.google.com/github/lavanya-shende/SRIP-2024/blob/main/Animals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install --upgrade --no-cache-dir gdown


  Attempting uninstall: gdown
    Found existing installation: gdown 4.7.3
    Uninstalling gdown-4.7.3:
      Successfully uninstalled gdown-4.7.3


In [7]:
# Import data from google drive
!gdown 12-0rljhOfL577E_HLiUZQIgBZLr_lGAK

Downloading...
From (original): https://drive.google.com/uc?id=12-0rljhOfL577E_HLiUZQIgBZLr_lGAK
From (redirected): https://drive.usercontent.google.com/download?id=12-0rljhOfL577E_HLiUZQIgBZLr_lGAK&confirm=t&uuid=40da6370-84d6-4ab5-ac6f-eacc0535e637
To: /content/archive (1).zip
100% 688M/688M [00:11<00:00, 60.8MB/s]


In [8]:
import zipfile

zip_ref = zipfile.ZipFile('/content/archive (1).zip')
zip_ref.extractall()
zip_ref.close()

In [9]:
#importing required libraries
import numpy as np
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import classification_report
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [10]:
import os

# Function to get subfolders in a directory
def get_subfolders(folder_path):
    subfolders = [f.name for f in os.scandir(folder_path) if f.is_dir()]
    return subfolders

# Path to the folder containing animal images
folder_path = '/content/animals/animals'
subfolders = get_subfolders(folder_path)

# print("Subfolders:")
names=[]
for subfolder in subfolders:
    names.append(subfolder)

In [ ]:
names

['donkey',
 'cat',
 'grasshopper',
 'pelecaniformes',
 'antelope',
 'caterpillar',
 'mouse',
 'badger',
 'flamingo',
 'koala',
 'pig',
 'starfish',
 'woodpecker',
 'pigeon',
 'hamster',
 'goose',
 'possum',
 'snake',
 'hedgehog',
 'squid',
 'deer',
 'wombat',
 'lobster',
 'mosquito',
 'lizard',
 'porcupine',
 'zebra',
 'ox',
 'moth',
 'coyote',
 'hyena',
 'rhinoceros',
 'fly',
 'crab',
 'raccoon',
 'dog',
 'orangutan',
 'squirrel',
 'hornbill',
 'goat',
 'otter',
 'seahorse',
 'oyster',
 'octopus',
 'rat',
 'beetle',
 'whale',
 'seal',
 'fox',
 'reindeer',
 'turkey',
 'sparrow',
 'gorilla',
 'horse',
 'boar',
 'okapi',
 'chimpanzee',
 'penguin',
 'bat',
 'hippopotamus',
 'kangaroo',
 'shark',
 'hummingbird',
 'swan',
 'bee',
 'eagle',
 'lion',
 'hare',
 'jellyfish',
 'sheep',
 'dragonfly',
 'parrot',
 'leopard',
 'dolphin',
 'wolf',
 'goldfish',
 'turtle',
 'owl',
 'crow',
 'bear',
 'duck',
 'panda',
 'tiger',
 'ladybugs',
 'bison',
 'butterfly',
 'elephant',
 'sandpiper',
 'cockroach'

In [11]:
#Define the classes
class_names = subfolders

#load and preprocess the dataset
data_dir = '/content/animals/animals'
img_height, img_width = 150, 150
batch_size = 32

# Use ImageDataGenerator for loading and preprocessing images
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

# Load and preprocess the dataset
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

Found 4320 images belonging to 90 classes.
Found 1080 images belonging to 90 classes.


In [12]:
#Organize the dataset for one-vs-rest classification
num_classes = len(class_names)



 Build a custom CNN model without using existing architectures like ResNet or DenseNet

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Define the model architecture
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Display the model summary
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 74, 74, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 36, 36, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 17, 17, 128)      

In [ ]:
# Step 4: Perform binary classification using existing architectures

from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout

# Load the pre-trained VGG16 model without the top (fully connected) layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# Freeze the convolutional base
base_model.trainable = False

# Create a new model on top of the pre-trained base model
model = Sequential([
    base_model,
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # binary classification, so use sigmoid activation
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(train_generator,
                    epochs=5,
                    validation_data=validation_generator)


Epoch 1/5
 31/135 [=====>........................] - ETA: 12:45 - loss: 0.2487 - accuracy: 0.9731

In [ ]:
train_generator.class_indices

{'antelope': 0,
 'badger': 1,
 'bat': 2,
 'bear': 3,
 'bee': 4,
 'beetle': 5,
 'bison': 6,
 'boar': 7,
 'butterfly': 8,
 'cat': 9,
 'caterpillar': 10,
 'chimpanzee': 11,
 'cockroach': 12,
 'cow': 13,
 'coyote': 14,
 'crab': 15,
 'crow': 16,
 'deer': 17,
 'dog': 18,
 'dolphin': 19,
 'donkey': 20,
 'dragonfly': 21,
 'duck': 22,
 'eagle': 23,
 'elephant': 24,
 'flamingo': 25,
 'fly': 26,
 'fox': 27,
 'goat': 28,
 'goldfish': 29,
 'goose': 30,
 'gorilla': 31,
 'grasshopper': 32,
 'hamster': 33,
 'hare': 34,
 'hedgehog': 35,
 'hippopotamus': 36,
 'hornbill': 37,
 'horse': 38,
 'hummingbird': 39,
 'hyena': 40,
 'jellyfish': 41,
 'kangaroo': 42,
 'koala': 43,
 'ladybugs': 44,
 'leopard': 45,
 'lion': 46,
 'lizard': 47,
 'lobster': 48,
 'mosquito': 49,
 'moth': 50,
 'mouse': 51,
 'octopus': 52,
 'okapi': 53,
 'orangutan': 54,
 'otter': 55,
 'owl': 56,
 'ox': 57,
 'oyster': 58,
 'panda': 59,
 'parrot': 60,
 'pelecaniformes': 61,
 'penguin': 62,
 'pig': 63,
 'pigeon': 64,
 'porcupine': 65,
 'pos

In [50]:
# Step 5: Restructure for 5-class classification

# Select five classes from the original 90 classes
five_classes = ['crab', 'crow', 'deer', 'zebra', 'wolf']  # Replace with your desired classes

import random

# Create a dictionary to map original class indices to new class indices
class_indices_map = {}
for cls in five_classes:
    if cls in train_generator.class_indices:
        class_indices_map[train_generator.class_indices[cls]] = len(class_indices_map)

# Assign other classes to one of the selected five classes randomly
for original_class_index in train_generator.class_indices.values():
    if original_class_index not in class_indices_map:
        class_indices_map[original_class_index] = random.randint(0, len(five_classes) - 1)

# Modify the labels for 5-class classification
five_class_train_labels = []
for label in train_generator.labels:
    five_class_train_labels.append(class_indices_map.get(label, random.randint(0, len(five_classes) - 1)))

five_class_validation_labels = []
for label in validation_generator.labels:
    five_class_validation_labels.append(class_indices_map.get(label, random.randint(0, len(five_classes) - 1)))

# Convert to numpy arrays
five_class_train_labels = np.array(five_class_train_labels)
five_class_validation_labels = np.array(five_class_validation_labels)

